In [16]:
import os
import shutil

import awkward as ak
import dask
import dask_awkward as dak
import mplhep as hep
import uproot
from coffea.dataset_tools import preprocess
from distributed import Client
from matplotlib import pyplot as plt

import egamma_tnp
from egamma_tnp import ElectronTagNProbeFromNTuples
from egamma_tnp.plot import plot_ratio
from egamma_tnp.utils.histogramming import save_hists

In [18]:
# Define the ntuples here 

fileset = {
    "events_2023D_Data" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/ec/tnpTuples/Prompt2023/data_2023D.root": "tnpEleTrig/fitter_tree",
        },
    },     
    "events_2023_Data" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/ec/tnpTuples/Prompt2023/data_2023C.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/ec/tnpTuples/Prompt2023/data_2023D.root": "tnpEleTrig/fitter_tree",
        },
    },
    "events_DY_NLO_2023_postBPix" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/ec/tnpTuples/Prompt2023/DY_NLO_2023postBPIX.root": "tnpEleTrig/fitter_tree",
        },
    },
    "events_DY_Winter24" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/ssaumya/Run3Winter24/TnPtuples/TnPTree_mc_Winter24.root": "tnpEleTrig/fitter_tree",
        },
    },    
    "events_2024C_Data" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-06-04/data_EGamma0_Run2024C.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-06-04/data_EGamma1_Run2024C.root": "tnpEleTrig/fitter_tree" 
        },
    },
    "events_2024D_Data" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-06-04/data_EGamma0_Run2024D.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-06-04/data_EGamma1_Run2024D.root": "tnpEleTrig/fitter_tree" 
        },
    }, 
    "events_2024E_Data" : {
        "files": {
#        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-06-04/data_EGamma0_Run2024E.root": "tnpEleTrig/fitter_tree",
#        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-06-04/data_EGamma1_Run2024E.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-02/data_EGamma0_Run2024Ev1.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-02/data_EGamma0_Run2024Ev2.root": "tnpEleTrig/fitter_tree",   
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-02/data_EGamma1_Run2024Ev1.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-02/data_EGamma1_Run2024Ev2.root": "tnpEleTrig/fitter_tree",               
        },
    },
    "events_2024F_Data" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-29/data_EGamma0_Run2024F.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-29/data_EGamma1_Run2024F.root": "tnpEleTrig/fitter_tree",    
        },
    },
    "events_2024G_Data" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-09-26/data_EGamma0_Run2024G.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-09-26/data_EGamma1_Run2024G.root": "tnpEleTrig/fitter_tree",  
        },
    },
    "events_2024H_Data" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-09-27/data_EGamma0_Run2024H.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-09-27/data_EGamma1_Run2024H.root": "tnpEleTrig/fitter_tree",  
        },
    }, 
    "events_2024I_Data" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-10-25/data_EGamma0_Run2024I.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-10-25/data_EGamma1_Run2024I.root": "tnpEleTrig/fitter_tree",  
        },
    },          
    "events_2024_Data" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-06-04/data_EGamma0_Run2024C.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-06-04/data_EGamma1_Run2024C.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-06-04/data_EGamma0_Run2024D.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-02/data_EGamma0_Run2024Ev1.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-02/data_EGamma0_Run2024Ev2.root": "tnpEleTrig/fitter_tree",   
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-02/data_EGamma1_Run2024Ev1.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-02/data_EGamma1_Run2024Ev2.root": "tnpEleTrig/fitter_tree",               
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-29/data_EGamma0_Run2024F.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-29/data_EGamma1_Run2024F.root": "tnpEleTrig/fitter_tree", 
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-09-26/data_EGamma0_Run2024G.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-09-26/data_EGamma1_Run2024G.root": "tnpEleTrig/fitter_tree",  
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-09-27/data_EGamma0_Run2024H.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-09-27/data_EGamma1_Run2024H.root": "tnpEleTrig/fitter_tree",  
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-10-25/data_EGamma0_Run2024I.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-10-25/data_EGamma1_Run2024I.root": "tnpEleTrig/fitter_tree",  
        },
    },
}

fileset_available, fileset_updated = preprocess(fileset, step_size=500_000, skip_bad_files=True)

In [19]:
# Define the HLT paths for wihich the efficiency plots are needed along with the last passing Filter

hlt_paths = {
    "Ele30": "passHLTEle30WPTightGsfTrackIsoFilter",
#    "Ele32": "passHLTEle32WPTightGsfTrackIsoFilter",
#    "Ele115": "passHLTEle115CaloIdVTGsfTrkIdTGsfDphiFilter",
#    "Ele135": "passHLTEle135CaloIdVTGsfTrkIdTGsfDphiFilter",
#    "Ele23Ele12Leg1": "passHLTEle23Ele12CaloIdLTrackIdLIsoVLTrackIsoLeg1Filter",
#    "Ele23Ele12Leg2": "passHLTEle23Ele12CaloIdLTrackIdLIsoVLTrackIsoLeg2Filter",
#    "DoubleEle33SeededLeg": "passHLTEle33CaloIdLMWPMS2Filter",
#    "DoubleEle33UnseededLeg": "passHLTDiEle33CaloIdLMWPMS2UnseededFilter",
}

# Check what the purpose of defining the plateau cuts is

plateau_cuts = {
    "Ele30": 35,
    "Ele32": 35,
    "Ele115": 120,
    "Ele135": 140,
    "Ele23Ele12Leg1": 25,
    "Ele23Ele12Leg2": 15,
    "DoubleEle33SeededLeg": 35,
    "DoubleEle33UnseededLeg": 35,
}

In [20]:
def runfilter(events):
    dataset = events.metadata["dataset"]
    print(dataset, "no run cut")
    return events

tnp = ElectronTagNProbeFromNTuples(fileset_available, list(hlt_paths.values()), cutbased_id="passingCutBasedTight122XV1", extra_filter=runfilter)

In [21]:
%%time

to_compute = {}

for name, trigger in hlt_paths.items():
    if name == "Ele115" or name == "Ele135":
        egamma_tnp.binning.set(
            "pt_bins",
            [
                5,
                10,
                15,
                20,
                22,
                26,
                28,
                30,
                32,
                34,
                36,
                38,
                40,
                45,
                50,
                60,
                80,
                100,
                105,
                110,
                115,
                120,
                125,
                130,
                135,
                140,
                145,
                150,
                200,
                250,
                300,
                350,
                400,
            ],
        )
    elif name == "Ele30":
        egamma_tnp.binning.set(
            "pt_bins",
            [
                5,
                10,
                12.5,
                15,
                17.5,
                20,
                22,
                26,
                28,
                30,
                32,
                34,
                36,
                38,
                40,
                45,
                50,
                60,
                80,
                100,
                150,
                250,
                400,
            ],
        )        
    else:
        egamma_tnp.binning.set(
            "pt_bins",
            [
                5,
                10,
                12,
                14,
                16,
                18,
                20,
                23,
                26,
                28,
                30,
                32,
                34,
                36,
                38,
                40,
                45,
                50,
                60,
                80,
                100,
                150,
                250,
                400,
            ],
        )
    plateau_cut = plateau_cuts[name]
    to_compute[name] = tnp.get_1d_pt_eta_phi_tnp_histograms(
        trigger,
        uproot_options={"allow_read_errors_with_report": True},
        eta_regions_pt={
            "barrel": [0.0, 1.4442],
            "endcap_loweta": [1.566, 2.0],
            "endcap_higheta": [2.0, 2.5],
        },
        plateau_cut=plateau_cut,
    )
    
dak.necessary_columns(to_compute)

events_2023D_Data no run cut
events_2023_Data no run cut
events_DY_NLO_2023_postBPix no run cut
events_DY_Winter24 no run cut
events_2024C_Data no run cut
events_2024D_Data no run cut
events_2024E_Data no run cut
events_2024F_Data no run cut
events_2024G_Data no run cut
events_2024H_Data no run cut
events_2024I_Data no run cut
events_2024_Data no run cut
CPU times: user 9.81 s, sys: 74.8 ms, total: 9.88 s
Wall time: 10 s


{'from-uproot-bb5160dfe796bc2736edceed004dffbe': frozenset({'el_eta',
            'el_phi',
            'el_pt',
            'el_q',
            'pair_mass',
            'passHLTEle30WPTightGsfTrackIsoFilter',
            'passingCutBasedTight122XV1',
            'tag_Ele_eta',
            'tag_Ele_pt',
            'tag_Ele_q'}),
 'from-uproot-21883be78c7efee4d69a0358d8f48f52': frozenset({'el_eta',
            'el_phi',
            'el_pt',
            'el_q',
            'pair_mass',
            'passHLTEle30WPTightGsfTrackIsoFilter',
            'passingCutBasedTight122XV1',
            'tag_Ele_eta',
            'tag_Ele_pt',
            'tag_Ele_q'}),
 'from-uproot-6eb80f6273bb05c054f54e6db6962022': frozenset({'el_eta',
            'el_phi',
            'el_pt',
            'el_q',
            'pair_mass',
            'passHLTEle30WPTightGsfTrackIsoFilter',
            'passingCutBasedTight122XV1',
            'tag_Ele_eta',
            'tag_Ele_pt',
            'tag_Ele_q'}),
 'fr

In [22]:
%%time

(out,) = dask.compute(to_compute)

#print(out)

CPU times: user 3min 44s, sys: 58.3 s, total: 4min 42s
Wall time: 12min 20s


In [23]:
# Make directory in afs/eos are to store the plots 

for dataset in out["Ele30"][1].keys():
    path = f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Winter24Checks/DataMC/{dataset}"
    if os.path.exists(path):
        shutil.rmtree(path)
    os.mkdir(path)

In [24]:
# Save histogram root files and json files in afs directory

for name, res in out.items():
    hists, report = res
    for dataset, report_arr in report.items():
        ak.to_json(
            report_arr,
            f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Winter24Checks/DataMC/{dataset}/{name}_report.json",
            num_readability_spaces=1,
            num_indent_spaces=4,
        )
    for dataset, hs in hists.items():
        save_hists(f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Winter24Checks/DataMC/{dataset}/{name}_hists.root", hs)

In [25]:
hep.style.use("CMS")
hep.style.use(
    {
        "figure.figsize": (6.4, 4.8),
        "font.size": 14,
        "legend.title_fontsize": 14,
        "savefig.bbox": "tight",
    }
)

In [26]:
# Fetch histograms from the root files for plotting

def get_histograms(path):
    with uproot.open(path) as file:
        hpt_barrel_pass = file["pt/barrel/passing"].to_hist()
        hpt_barrel_fail = file["pt/barrel/failing"].to_hist()
        hpt_endcap_loweta_pass = file["pt/endcap_loweta/passing"].to_hist()
        hpt_endcap_loweta_fail = file["pt/endcap_loweta/failing"].to_hist()
        hpt_endcap_higheta_pass = file["pt/endcap_higheta/passing"].to_hist()
        hpt_endcap_higheta_fail = file["pt/endcap_higheta/failing"].to_hist()
        hpt_combined_pass = hpt_barrel_pass + hpt_endcap_loweta_pass + hpt_endcap_higheta_pass
        hpt_combined_fail = hpt_barrel_fail + hpt_endcap_loweta_fail + hpt_endcap_higheta_fail
        
        heta_entire_pass = file["eta/entire/passing"].to_hist()
        heta_entire_fail = file["eta/entire/failing"].to_hist()

        hphi_entire_pass = file["phi/entire/passing"].to_hist()
        hphi_entire_fail = file["phi/entire/failing"].to_hist()

    return (
        hpt_barrel_pass,
        hpt_barrel_fail,
        hpt_endcap_loweta_pass,
        hpt_endcap_loweta_fail,
        hpt_endcap_higheta_pass,
        hpt_endcap_higheta_fail,
        hpt_combined_pass,
        hpt_combined_fail,
        heta_entire_pass,
        heta_entire_fail,
        hphi_entire_pass,
        hphi_entire_fail,
    )

In [27]:
# Plot set-up: legends, tick marks, labelling of x and y-axis

def pt_low_threshold_plot_setup(**legend_kwargs):
    plt.xlim(10, 400)
    plt.ylim(0, 1.2)
    plt.xlabel(r"Offline electron $P_T$ [GeV]")
    plt.ylabel(r"Efficiency")
    plt.xscale("log")
    plt.xticks([10, 100], [10, 100])
    plt.xticks(
        [20, 30, 40, 50, 60, 70, 80, 90, 200, 300, 400],
        [20, 30, 40, 50, None, None, None, None, 200, 300, 400],
        minor=True,
    )
    plt.legend(**legend_kwargs) if legend_kwargs else plt.legend()


def pt_high_threshold_plot_setup(**legend_kwargs):
    plt.xlim(10, 400)
    plt.ylim(0, 1.2)
    plt.xlabel(r"Offline electron $P_T$ [GeV]")
    plt.ylabel(r"Efficiency")
    plt.legend(**legend_kwargs) if legend_kwargs else plt.legend()


def eta_plot_setup(**legend_kwargs):
    plt.xlim(-2.5, 2.5)
    plt.ylim(0, 1.2)
    plt.xlabel(r"Offline electron $\eta$")
    plt.ylabel(r"Efficiency")
    plt.legend(**legend_kwargs) if legend_kwargs else plt.legend()


def phi_plot_setup(**legend_kwargs):
    plt.xlim(-3.32, 3.32)
    plt.ylim(0, 1.2)
    plt.xlabel(r"Offline electron $\phi$")
    plt.ylabel(r"Efficiency")
    plt.legend(**legend_kwargs) if legend_kwargs else plt.legend()

In [28]:
lumis = {
    "2023D": 9.525,
    "2024C": 7.416594091,
    "2024D": 7.889161918,
    "2024Ev1": 6.279524894,
    "2024Ev2": 5.040346798,
    "2024F": 26.313333799,
    "2024G": 17.604084255,
}

In [42]:
for tala in list(hlt_paths.keys()):
#    for data_period in ["events_2023D_Data"]:
    for data_period in ["events_2024_Data"]:
        for mc_dataset in ["events_DY_NLO_2023_postBPix"]: ## data_period = reference, mc_dataset = target
#        for mc_dataset in ["events_DY_Winter24"]:
            tocompare = [data_period, mc_dataset]
            run = []
            for folder in tocompare:
                run.append(folder.split("_", 2)[2][3:] if "data" in folder else folder.split("_", 1)[1])
            threshold = tala
            if threshold == "Ele32" or threshold == "Ele30":
                suffix = "WPTight_Gsf"
            elif threshold == "Ele115" or threshold == "Ele135":
                suffix = "CaloIdVT_GsfTrkIdT"
            elif threshold == "DoubleEle33SeededLeg":
                suffix = "CaloIdL_MW\nSeeded leg"
            elif threshold == "DoubleEle33UnseededLeg":
                suffix = "CaloIdL_MW\nUnseeded leg"
            elif threshold == "Ele23Ele12Leg1":
                suffix = "CaloIdL_TrackIdL_IsoVL Leg1"
            elif threshold == "Ele23Ele12Leg2":
                suffix = "CaloIdL_TrackIdL_IsoVL Leg2"
            else:
                raise ValueError("Couldn't find proper trigger name")

            plateau_cut_dict = {
                "Ele30": 35,
                "Ele32": 35,
                "Ele115": 120,
                "Ele135": 140,
                "Ele23Ele12Leg1": 25,
                "Ele23Ele12Leg2": 15,
                "DoubleEle33SeededLeg": 35,
                "DoubleEle33UnseededLeg": 35,
            }
            plateau_cut = plateau_cut_dict[threshold]

            filename = threshold
            threshold = threshold.replace("Leg1", "").replace("Leg2", "").replace("SeededLeg", "").replace("UnseededLeg", "")

            plottype = "pt_high_threshold" if threshold == "Ele115" or threshold == "Ele135" else "pt_low_threshold"
            title = f"HLT_{threshold}_{suffix}"
            lumi = []
            for r in run:
                try:
                    l = lumis[r]
                except KeyError:
                    if r == "2022":
                        l = lumis["2022C"] + lumis["2022D"] + lumis["2022E"] + lumis["2022F"] + lumis["2022G"]
                    elif r == "2023":
                        l = lumis["2023B"] + lumis["2023C"] + lumis["2023D"]
                    else:
                        l = "X"
                if not isinstance(l, str):
                    l = round(l, 1)
                lumi.append(l)

            year = []
            for r in run:
                if "2022" in r:
                    year.append("2022")
                elif "2023" in r:
                    year.append("2023")
                else:
                    year.append("2024")

            #if mc_dataset.startswith("mc_"):
            #    rlabel = f"{lumi[0]} $fb^{{-1}}$, {year[0]} (13.6 TeV)"
            #else:
            #    rlabel = f"{lumi[0]} $fb^{{-1}}$, {year[0]} (13.6 TeV) - {lumi[1]} $fb^{{-1}}$, {year[1]} (13.6 TeV)"
            rlabel = "(13.6 TeV)"

            if mc_dataset == "events_DY_NLO_2023_postBPix":
            #if mc_dataset == "events_DY_Winter24":
                eff2_kwargs = {"color": "#e42536"} # red
                #eff2_kwargs = {"color": "#5790fc"}  # Blue
            else:
                eff2_kwargs = {"color": "#5790fc"}

            (
                hpt_barrel_pass1,
                hpt_barrel_all1,
                hpt_endcap_loweta_pass1,
                hpt_endcap_loweta_all1,
                hpt_endcap_higheta_pass1,
                hpt_endcap_higheta_all1,
                hpt_combined_pass1,
                hpt_combined_all1,
                heta_entire_pass1,
                heta_entire_all1,
                hphi_entire_pass1,
                hphi_entire_all1,
            ) = get_histograms(f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Winter24Checks/DataMC/{tocompare[0]}/{filename}_hists.root")

            (
                hpt_barrel_pass2,
                hpt_barrel_all2,
                hpt_endcap_loweta_pass2,
                hpt_endcap_loweta_all2,
                hpt_endcap_higheta_pass2,
                hpt_endcap_higheta_all2,
                hpt_combined_pass2,
                hpt_combined_all2,
                heta_entire_pass2,
                heta_entire_all2,
                hphi_entire_pass2,
                hphi_entire_all2,
            ) = get_histograms(f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Winter24Checks/DataMC/{tocompare[1]}/{filename}_hists.root")

            plot_ratio(
                hpt_barrel_pass1,
                hpt_barrel_all1,
                hpt_barrel_pass2,
                hpt_barrel_all2,
                label1=f"{run[0]}",
                label2=f"{run[1]}",
                denominator_type="failing",
                plottype=plottype,
                #figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Winter24Checks/DataMC/{filename}_{run[0]}_vs_{run[1]}_HLT_eff_barrel_pt.png",
                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Winter24Checks/DataMC/{filename}_{run[0]}_vs_{run[1]}_HLT_eff_barrel_pt.pdf",
                legend_kwargs={"title": f"{title}\n$0.00 < |\eta| < 1.44$"},
                cms_kwargs={"loc": 1, "rlabel": rlabel},
                eff2_kwargs=eff2_kwargs,
                efficiency_label="L1T + HLT Efficiency",
            )

            plot_ratio(
                hpt_endcap_loweta_pass1,
                hpt_endcap_loweta_all1,
                hpt_endcap_loweta_pass2,
                hpt_endcap_loweta_all2,
                label1=f"{run[0]}",
                label2=f"{run[1]}",
                denominator_type="failing",
                plottype=plottype,
                #figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Winter24Checks/DataMC/{filename}_{run[0]}_vs_{run[1]}_HLT_eff_endcap_loweta_pt.png",
                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Winter24Checks/DataMC/{filename}_{run[0]}_vs_{run[1]}_HLT_eff_endcap_loweta_pt.pdf",
                legend_kwargs={"title": f"{title}\n$1.57 < |\eta| < 2.00$"},
                cms_kwargs={"loc": 1, "rlabel": rlabel},
                eff2_kwargs=eff2_kwargs,
                efficiency_label="L1T + HLT Efficiency",
            )

            plot_ratio(
                hpt_endcap_higheta_pass1,
                hpt_endcap_higheta_all1,
                hpt_endcap_higheta_pass2,
                hpt_endcap_higheta_all2,
                label1=f"{run[0]}",
                label2=f"{run[1]}",
                denominator_type="failing",
                plottype=plottype,
                #figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Winter24Checks/DataMC/{filename}_{run[0]}_vs_{run[1]}_HLT_eff_endcap_higheta_pt.png",
                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Winter24Checks/DataMC/{filename}_{run[0]}_vs_{run[1]}_HLT_eff_endcap_higheta_pt.pdf",
                legend_kwargs={"title": f"{title}\n$2.00 < |\eta| < 2.50$"},
                cms_kwargs={"loc": 1, "rlabel": rlabel},
                eff2_kwargs=eff2_kwargs,
                efficiency_label="L1T + HLT Efficiency",
            )

            plot_ratio(
                hpt_combined_pass1,
                hpt_combined_all1,
                hpt_combined_pass2,
                hpt_combined_all2,
                label1=f"{run[0]}",
                label2=f"{run[1]}",
                denominator_type="failing",
                plottype=plottype,
                #figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Winter24Checks/DataMC/{filename}_{run[0]}_vs_{run[1]}_HLT_eff_combined_pt.png",
                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Winter24Checks/DataMC/{filename}_{run[0]}_vs_{run[1]}_HLT_eff_combined_pt.pdf",
                legend_kwargs={"title": f"{title}\n$0.00 < |\eta| < 1.44$ or $1.57 < |\eta| < 2.50$"},
                cms_kwargs={"loc": 1, "rlabel": rlabel},
                eff2_kwargs=eff2_kwargs,
                efficiency_label="L1T + HLT Efficiency",
            )

            plot_ratio(
                heta_entire_pass1,
                heta_entire_all1,
                heta_entire_pass2,
                heta_entire_all2,
                label1=f"{run[0]}",
                label2=f"{run[1]}",
                denominator_type="failing",
                plottype="eta",
                #figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Winter24Checks/DataMC/{filename}_{run[0]}_vs_{run[1]}_HLT_eff_eta.png",
                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Winter24Checks/DataMC/{filename}_{run[0]}_vs_{run[1]}_HLT_eff_eta.pdf",
                legend_kwargs={"title": f"{title}\n$0.00 < |\eta| < 2.50$\nProbe electron $P_T> {plateau_cut}$ GeV"},
                cms_kwargs={"loc": 1, "rlabel": rlabel},
                eff2_kwargs=eff2_kwargs,
                efficiency_label="L1T + HLT Efficiency",
            )

            plot_ratio(
                hphi_entire_pass1,
                hphi_entire_all1,
                hphi_entire_pass2,
                hphi_entire_all2,
                label1=f"{run[0]}",
                label2=f"{run[1]}",
                denominator_type="failing",
                plottype="phi",
                #figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Winter24Checks/DataMC/{filename}_{run[0]}_vs_{run[1]}_HLT_eff_phi.png",
                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Winter24Checks/DataMC/{filename}_{run[0]}_vs_{run[1]}_HLT_eff_phi.pdf",
                legend_kwargs={"title": f"{title}\n$0.00 < |\eta| < 2.50$\nProbe electron $P_T> {plateau_cut}$ GeV"},
                cms_kwargs={"loc": 1, "rlabel": rlabel},
                eff2_kwargs=eff2_kwargs,
                efficiency_label="L1T + HLT Efficiency",
            )